#### Arranging Dataset

In [ ]:
# import os
# import shutil
# from pathlib import Path

# # Source folders
# cam1_dir = r"C:\Sem_5\Portfolio1\Task-2\dataset\dataset\Cam1"
# cam2_dir = r"C:\Sem_5\Portfolio1\Task-2\dataset\dataset\Cam2"

# # Target
# output_root = r"C:\Sem_5\Portfolio1\Task-2\reid_dataset"
# train_dir = os.path.join(output_root, "train")

# os.makedirs(train_dir, exist_ok=True)

# def copy_to_person_folder(img_path, pid, camid):
#     target_dir = os.path.join(train_dir, f"person_{pid:04d}")
#     os.makedirs(target_dir, exist_ok=True)
#     img_name = f"cam{camid}_{Path(img_path).name}"
#     shutil.copy(img_path, os.path.join(target_dir, img_name))

# # Process Cam1
# for filename in os.listdir(cam1_dir):
#     if filename.startswith("person_"):
#         pid = int(filename.split("_")[1])
#         copy_to_person_folder(os.path.join(cam1_dir, filename), pid, 1)

# # Process Cam2
# for filename in os.listdir(cam2_dir):
#     if filename.startswith("person_"):
#         pid = int(filename.split("_")[1])
#         copy_to_person_folder(os.path.join(cam2_dir, filename), pid, 2)

# print("✅ Dataset prepared in TorchReID format.")


### Model Finetuning

In [1]:
import os
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
import torchvision.transforms as T
from torchreid.data import ImageDataset, register_image_dataset
from torchreid import models
from torchreid.utils import set_random_seed

# -----------------------------
# Register Your Dataset (with ID remapping)
# -----------------------------
class YourDataset(ImageDataset):
    dataset_dir = 'reid_dataset'

    def __init__(self, root='', **kwargs):
        self.dataset_dir = os.path.join(root, self.dataset_dir)
        train_dir = os.path.join(self.dataset_dir, 'train')
        train = self._process_dir(train_dir)

        # Dummy query/gallery to prevent TorchReID error
        query = [(train[0][0], train[0][1], train[0][2], 0)]
        gallery = [(train[0][0], train[0][1], train[0][2], 0)]

        super().__init__(train, query, gallery, **kwargs)

    def _process_dir(self, dir_path):
        data = []
        pid_container = []

        # Step 1: Collect PIDs
        for pid_folder in os.listdir(dir_path):
            pid_path = os.path.join(dir_path, pid_folder)
            if not os.path.isdir(pid_path): continue
            pid = int(pid_folder.replace('person_', ''))
            pid_container.append(pid)

        pid2label = {pid: idx for idx, pid in enumerate(sorted(pid_container))}

        # Step 2: Build data with relabeled PIDs
        for pid_folder in os.listdir(dir_path):
            pid_path = os.path.join(dir_path, pid_folder)
            if not os.path.isdir(pid_path): continue
            pid = int(pid_folder.replace('person_', ''))
            label = pid2label[pid]
            for fname in os.listdir(pid_path):
                if not fname.lower().endswith('.jpg'): continue
                camid = 0 if 'cam1' in fname.lower() else 1
                img_path = os.path.join(pid_path, fname)
                data.append((img_path, label, camid))
        return data

register_image_dataset('yourdataset', YourDataset)

# -----------------------------
# Setup
# -----------------------------
set_random_seed(42)
root = r"C:\Sem_5\Portfolio1\Task-2"
dataset = YourDataset(root=root)
num_classes = len(set([x[1] for x in dataset.train]))

transform = T.Compose([
    T.Resize((256, 128)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# -----------------------------
# Model
# -----------------------------
model = models.build_model(
    name='osnet_ain_x1_0',
    num_classes=num_classes,
    loss='softmax',
    pretrained=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("✅ Training on:", device)

# -----------------------------
# DataLoader
# -----------------------------
class ReidTrainWrapper(torch.utils.data.Dataset):
    def __init__(self, dataset, transform):
        self.dataset = dataset.train
        self.transform = transform

    def __getitem__(self, idx):
        img_path, pid, camid, _ = self.dataset[idx]
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)
        return img, pid

    def __len__(self):
        return len(self.dataset)

train_loader = DataLoader(
    ReidTrainWrapper(dataset, transform),
    batch_size=16,
    shuffle=True,
    num_workers=0
)

# -----------------------------
# Training Loop
# -----------------------------
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
model.train()

for epoch in range(1, 101):  # Adjust epoch count as needed
    total_loss = 0
    print(f"\nEpoch {epoch}")
    for imgs, pids in tqdm(train_loader, desc="Training", leave=False):
        imgs = imgs.to(device)
        pids = pids.to(device)

        outputs = model(imgs)
        loss = F.cross_entropy(outputs, pids)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"✅ Epoch {epoch} - Loss: {total_loss:.4f}")



=> Loaded YourDataset
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |     9 |      763 |         2
  query    |     1 |        1 |         1
  gallery  |     1 |        1 |         1
  ----------------------------------------


C:\Softwares\Anaconda\envs\clouds_modeling\lib\site-packages\torchreid\models\osnet_ain.py:492: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(cached_

Successfully loaded imagenet pretrained weights from "C:\Users\korin/.cache\torch\checkpoints\osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
✅ Training on: cuda

Epoch 1


✅ Epoch 1 - Loss: 49.5420

Epoch 2


✅ Epoch 2 - Loss: 7.1234

Epoch 3


...

✅ Epoch 96 - Loss: 0.0048

Epoch 97


✅ Epoch 97 - Loss: 0.0045

Epoch 98


✅ Epoch 98 - Loss: 0.0035

Epoch 99


✅ Epoch 99 - Loss: 0.0031

Epoch 100


✅ Epoch 100 - Loss: 0.0037


In [2]:
torch.save(model.state_dict(), "reid_osnet.pt")


### Test on some images

In [2]:
import torch
from PIL import Image
import torchvision.transforms as T
from torch.nn.functional import cosine_similarity

# Your two test image paths (modify if needed)
img1_path = r"C:\Sem_5\Portfolio1\Task-2\reid_dataset\train\person_0005\cam2_person_5_54.jpg"
img2_path = r"C:\Sem_5\Portfolio1\Task-2\reid_dataset\train\person_0005\cam2_person_5_53.jpg"

# Image transform (same as training)
transform = T.Compose([
    T.Resize((256, 128)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
# Load and process images
def get_embedding(path):
    img = Image.open(path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        embedding = model(img_tensor)
    return embedding

# Get embeddings
embedding1 = get_embedding(img1_path)
embedding2 = get_embedding(img2_path)

# Compute cosine similarity
similarity = cosine_similarity(embedding1, embedding2).item()
print(f"🔍 Cosine Similarity between two images: {similarity:.4f}")


NameError: name 'model' is not defined

<h1 style="text-align:center;">The End</h1>